In [4]:
# COMPLETE AI FAIRNESS AUDIT IMPLEMENTATION

# 1. SETUP AND IMPORTS
!pip install aif360 pandas numpy matplotlib seaborn scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# 2. DATA PREPARATION
def load_compas_data():
    """Load and preprocess COMPAS dataset"""
    df = pd.read_csv('compas-scores-two-years.csv')

    # Filter as per ProPublica's analysis
    df = df[(df['days_b_screening_arrest'] <= 30) &
            (df['days_b_screening_arrest'] >= -30) &
            (df['is_recid'] != -1) &
            (df['c_charge_degree'] != 'O') &
            (df['score_text'] != 'N/A')]

    # Prepare features and target
    features = df.drop(['two_year_recid', 'race', 'sex', 'age', 'c_charge_desc'], axis=1)
    X = pd.get_dummies(features)
    y = np.array(df['two_year_recid'])  # Recidivism outcome
    protected = np.array(df['race'] == 'African-American', dtype=int)  # 1 if African-American

    return X, y, protected

# 3. FAIRNESS ANALYSIS
def analyze_fairness(dataset):
    """Calculate and display fairness metrics"""
    metric = BinaryLabelDatasetMetric(
        dataset,
        unprivileged_groups=[{'race': 1}],
        privileged_groups=[{'race': 0}]
    )

    print("\n" + "="*50)
    print("FAIRNESS METRICS ANALYSIS")
    print("="*50)
    print(f"Disparate Impact: {metric.disparate_impact():.3f} (ideal: 1.0)")
    print(f"Statistical Parity Difference: {metric.statistical_parity_difference():.3f} (ideal: 0)")
    print(f"Mean Difference: {metric.mean_difference():.3f} (ideal: 0)")

    return metric

# 4. BIAS MITIGATION
def mitigate_bias(train_data):
    """Apply reweighing to mitigate bias"""
    RW = Reweighing(
        unprivileged_groups=[{'race': 1}],
        privileged_groups=[{'race': 0}]
    )
    return RW.fit_transform(train_data)

# 5. MODEL TRAINING AND EVALUATION
def train_and_evaluate(X_train, y_train, X_test, y_test):
    """Train logistic regression model and evaluate performance"""
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("\n" + "="*50)
    print("MODEL PERFORMANCE")
    print("="*50)
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    return model, y_pred

# 6. VISUALIZATION
def plot_fairness_comparison(original_metrics, mitigated_metrics):
    """Visualize fairness metrics before/after mitigation"""
    metrics_df = pd.DataFrame({
        'Original': original_metrics,
        'Mitigated': mitigated_metrics
    }, index=['Disparate Impact', 'Statistical Parity Difference'])

    fig, ax = plt.subplots(figsize=(10, 6))
    metrics_df.plot.bar(ax=ax)
    ax.axhline(y=0, color='k', linestyle='--')
    ax.axhline(y=1, color='g', linestyle='--', alpha=0.3)
    ax.set_title('Fairness Metrics Comparison', pad=20)
    ax.set_ylabel('Metric Value')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig('fairness_comparison.png', dpi=300)
    plt.show()

# MAIN EXECUTION
if __name__ == "__main__":
    # Load and prepare data
    X, y, protected = load_compas_data()

    # Create AIF360 dataset
    dataset = BinaryLabelDataset(
        df=pd.concat([X, pd.Series(y, name='two_year_recid')], axis=1),
        label_names=['two_year_recid'],
        protected_attribute_names=['race'],
        privileged_protected_attributes=[0]
    )

    # Split data
    train, test = dataset.split([0.7], shuffle=True, seed=42)

    # Analyze original fairness
    print("\nOriginal Data Fairness:")
    orig_metrics = analyze_fairness(train)

    # Mitigate bias
    train_transformed = mitigate_bias(train)
    print("\nAfter Bias Mitigation:")
    trans_metrics = analyze_fairness(train_transformed)

    # Train models
    ## Original data
    X_train_orig = train.features
    y_train_orig = train.labels.ravel()
    X_test = test.features
    y_test = test.labels.ravel()

    ## Transformed data
    train_trans_df = train_transformed.convert_to_dataframe()[0]
    X_train_trans = train_trans_df.drop('two_year_recid', axis=1)
    y_train_trans = train_trans_df['two_year_recid']

    # Train and evaluate models
    print("\nOriginal Data Model:")
    orig_model, orig_pred = train_and_evaluate(X_train_orig, y_train_orig, X_test, y_test)

    print("\nMitigated Data Model:")
    trans_model, trans_pred = trans_model, trans_pred = train_and_evaluate(X_train_trans, y_train_trans, X_test, y_test)

    # Compare fairness metrics
    plot_fairness_comparison(
        [orig_metrics.disparate_impact(), orig_metrics.statistical_parity_difference()],
        [trans_metrics.disparate_impact(), trans_metrics.statistical_parity_difference()]
    )

    # Generate classification fairness metrics
    test_pred = test.copy()
    test_pred.labels = trans_pred.reshape(-1, 1)

    class_metric = ClassificationMetric(
        test,
        test_pred,
        unprivileged_groups=[{'race': 1}],
        privileged_groups=[{'race': 0}]
    )

    print("\n" + "="*50)
    print("CLASSIFICATION FAIRNESS METRICS")
    print("="*50)
    print(f"False Positive Rate Difference: {class_metric.false_positive_rate_difference():.3f} (ideal: 0)")
    print(f"False Negative Rate Difference: {class_metric.false_negative_rate_difference():.3f} (ideal: 0)")
    print(f"Equal Opportunity Difference: {class_metric.equal_opportunity_difference():.3f} (ideal: 0)")
    print(f"Average Odds Difference: {class_metric.average_odds_difference():.3f} (ideal: 0)")

KeyError: 'days_b_screening_arrest'

In [3]:
# You can download it programmatically using:
import urllib.request
import zipfile

url = "https://www.propublica.org/datastore/dataset/6b259406-68c5-49e5-a83e-6a4dbd5ef04b/resource/52fd8e92-571c-4c29-9a4e-80def71b3682/download/compas-scores-two-years.csv"
urllib.request.urlretrieve(url, "compas-scores-two-years.csv")

('compas-scores-two-years.csv', <http.client.HTTPMessage at 0x7f15f7a65350>)